# How to control the verbosity of sos output and the number of running jobs

* **Difficulty level**: easy
* **Time need to lean**: 10 minutes or less
* **Key points**:
  * `-v` controls logging level from `error`, `warning`, `info` to `debug`
  * `-v1` introduces a special text-based progress bar
  * `%set -v` sets a persistent logging level for the entire notebook
  * Environment variable `DBG_DEBUG` writes topic-specific information to `~/sos_debug.log`
  * `-j` controls number of concurrent processes

## Logging level

SoS uses a logging system to output all sorts of information during the execution of workflows. The amount of output can be controlled by logging level, which can be `error` (0), `warning` (1), `info` (2), and `debug` (3). The default logging level for SoS is `info`.

For example, logging at `info` level would produce message indicating the steps executed and input output files, while `-v0` will suppress all but error messages. It is worth noting that if you are using SoS Notebook, all logging messages will be sent to the side panel if the side panel is open.

In [2]:
!touch a.txt

%run -v3
[10]
[20]
input: 'a.txt'
[30]
[40]

INFO: Running 10 :

INFO: Running 20 :

INFO: Running 30 :

INFO: Running 40 :

INFO: Workflow default (ID=da9f62e48e2b6ead) is executed successfully with 4 completed steps.

<div class="bs-callout bs-callout-primary" role="alert">
    <h4>Text-based progress bar with <code>-v1</code></h4>
    <p>If you have a large workflow with lots of steps, the default output can be overwhelming. In this case you can use <code>-v1</code>, which not only suppress most of the `INFO` messages, but also introduces a text-based progress bar from which you can learn the progress of the workflow easily.</p>
</div>

In [3]:
!touch a.txt

%run -v1
[10]
[20]
input: 'a.txt'
[30]
[40]

[####] 4 steps processed (4 jobs completed)


## `SOS_DEBUG` and `~/sos_debug.log` 

This option is designed for developers but if you would like to peak into what SoS is actually doing, you can set an environment variable `SOS_DEBUG`, with `,` separated values

* `ALL`: all debug messages
* Topics such as `CONTROLLER`, `TASK`, `WORKER`, `EXECUTOR` to keep only debug information of specific topics. The actual values can vary so you should check the output of command `sos run -h` for acceptable topics.
* A filename with an extension. If you include a filename in `SOS_DEBUG` (e.g. `SOS_DEBUG=TASK,tasks.log`), the debug output will be saved to specified files. If no file is specified, the debug information will be saved to `~/sos_debug.log`.

Again, `SOS_DEBUG` can be used to generate a large amount of information for debugging purposes. You usually do not need to use it but if you encounter some problems with SoS and would like to submit a bug report, it is a good idea to upload a log file with your workflow executed under 

```
export SOS_DEBUG=ALL,bug_report.log
```



## Control the number of running jobs

<div class="bs-callout bs-callout-primary" role="alert">
    <h4>Control number of workers with option <code>-j</code></h4>
    <p>Option <code>-j</code> sets the number of workers for SoS. The default value is half of the number of processes.</p>
</div>

SoS dispatch steps, substeps, and nested workflows to dedicated workers. The maximum number of concurrent workers can be controlled by the `-j` option. Although the default value, which is half of the number of processes of the computer, is good most of the time, you might want to control the number of workers depending on your running environment (e.g. on a shared server) or on the size of your workflow. Note that `-j` specifies the number of workers so the total number of processes started by `sos` can be at most `n + 1` with option `-j n`.

<div class="bs-callout bs-callout-warning" role="alert">
    <h4>Dedicated workers for blocking nested workflows</h4>
    <p>Ocassionally you might see more than <code>n + 1</code> SoS processes when you execute a workflow with option `-j n`. This is because the workflow consists of blocking nested workflows, namely nested workflows not as the last statement of a step. For example, the nested workflow `A` in the following SoS step has to be executed in a blocking fashion before `some_other_computation` can be done,</p>
    <pre>
        [10]
        sos_run('A')
        some_other_computation()
        </pre>
    <p>So SoS spawns a separate process for nested workflow <code>A</code> so that it will not affect the concurrent execution of other parts of the workflow.</p>    
</div>